Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


In [421]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
from anytree import Node, RenderTree, search

## The *Nim* and *Nimply* classes

In [422]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [423]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None: #k is the max number of object that can be removed from a line
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [424]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)    

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [425]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [426]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [427]:
def enrico(state: Nim) -> Nimply: #my strategy
    data = cook_status(state)
    if data["active_rows_number"] == 1:
        return (data["longest_row"], state._rows[data["longest_row"]])
    else:
        return (data["shortest_row"], state._rows[data["shortest_row"]])     

In [428]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [429]:
NUM_MATCHES = 1
NIM_SIZE = 3

<h3>MinMax Agent</h3>

In [430]:
class Agent():
    def __init__(self, state: Nim):
        self.nim = state
        self.tree = None
        
    def createTree(self):
        creation = True
        self.tree = Node('root', value = list(self.nim.rows))
        parents = [self.tree]
        childs = []
        level = 1
        my_turn = True
        while creation:
            #print("level:", level)
            #print("parents: ", parents)
            count_parents_no_child = 0
            for parent in parents:
                #print(parent)
                moves = [(r, o) for r, c in enumerate(parent.value) for o in range(1, c + 1) ]
                #print("moves", moves)
                if len(moves) == 0:
                    count_parents_no_child+=1
                else:    
                    for ply in moves:
                        aus = parent.value.copy()
                        
                        aus[ply[0]] = (aus[ply[0]] - ply[1])
                        child = Node(level,value=aus, after_me=level%2, parent=parent)
                        childs.append(child)
            if count_parents_no_child == len(parents):
                creation = False #no more node to explore
            level+=1
            my_turn != my_turn
            parents = childs  
            childs = []
        #print(RenderTree(self.tree))    

    def findMybestNextMove(self, root: Node) -> Nimply:
        d = search.findall(root, filter_=lambda node: node.value == [0]*NIM_SIZE and node.after_me == 1)
        best = min(d, key=lambda n: n.name)[0]
        

        return pure_random(self.nim)

    def minmax(self) -> Nimply:
        
        if self.tree == None:
            self.createTree()       #create tree for first time
        else:
            for child in self.tree.children:            #updaate tree root with the current nim state
                if child.value == list(self.nim.rows):
                    self.tree = child

        self.findMybestNextMove(self.tree)            
        
        return pure_random(self.nim)

In [431]:

def evaluate(opponent: Callable) -> float:
    won = 0
    last_player_start = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        agent = Agent(nim)
        player = 1 - last_player_start
        last_player_start = player
        while nim:
            if player == 0:
                ply = opponent(nim)
            else:
                ply = agent.minmax()
            nim.nimming(ply)
            player = 1 - player
        if player == 0:
            won += 1

    return won / NUM_MATCHES #percentage of match won agaist the opponent

In [432]:
evaluate(enrico) 

Node('/root/1/2/3', after_me=1, value=[0, 0, 0])
Node('/root/1/2/3', after_me=1, value=[0, 0, 0])
Node('/root/1/2/3', after_me=1, value=[0, 0, 0])


1.0